imports

In [1]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('..'))


In [8]:
import numpy as np
import pandas as pd
import torch

import src.preprocessing as pre
import src.utils as utils
import src.table as table
import src.graphs as graphs
from src.model import HarvestModel

initialize shit

In [3]:
train, test, mappings, meta = pre.separate_year('../data/meta.json','../data/y.csv','../data/mappings.json')

In [9]:
model = HarvestModel()

In [41]:
model.load_state_dict(torch.load("../app/model1.pth"))

<All keys matched successfully>

In [12]:

preds = utils.predict_gridded_harvest(model,test)
actuals = test.Y.detach().numpy()
production_plan = pre.decode(meta,mappings).reset_index(drop=True)
idx_dict = table.create_indices_dict(production_plan)

transplant_weeks = production_plan.WeekTransplanted.to_numpy()
year_preds, year_actuals = table.season_shift(transplant_weeks,preds,actuals)
year_preds.shape,year_actuals.shape

((139, 20, 51), (139, 51))

In [13]:
preds.shape, actuals.shape

((139, 20, 20), (139, 20))

full season totals

In [14]:
fig1 = graphs.graph_harvest(year_preds,year_actuals)

In [15]:
fig1.show()

received only harvest cycle

In [16]:
nonzero_indices = np.any(actuals != 0, axis=1)
actuals_received = actuals[nonzero_indices]
preds_received = preds[nonzero_indices]

fig2 = graphs.graph_harvest(preds_received,actuals_received)

In [17]:
actuals_received.shape

(139, 20)

In [18]:
fig4 = graphs.graph_harvest_cumsum(preds_received,actuals_received)

In [19]:
fig2.show()

In [20]:
fig4.show()

total harvest cycle

In [21]:
fig3 = graphs.graph_harvest(preds,actuals)

In [22]:
fig5 = graphs.graph_harvest_cumsum(preds,actuals)

In [23]:
fig3.show()

In [24]:
fig5.show()

In [25]:
summary_class = production_plan.groupby(['Class','Type']).agg(
    {'Ha':'sum',
     'WeekTransplanted':['min','max','nunique'],
     'Ranch':'nunique'
})

In [26]:
types = summary_class.index.get_level_values('Type')

In [27]:
preds_all_types, actuals_all_types = table.filter_preds(preds,actuals,'Type',types,idx_dict)
preds_all_types.shape,actuals_all_types.shape

((7, 20, 20), (7, 20))

In [39]:
actuals_received = actuals_all_types.sum(axis=1)
final_preds = preds_all_types.sum(axis=2)[:,-1]
initial_preds = preds_all_types.sum(axis=2)[:,6]

table summary, important stats, % expected, amount received, adjusted %, initial predictions, final predictions, totals by class, tomato type

In [40]:
summary_class['KilosReceived'] = actuals_received
summary_class['FinalPreds'] = final_preds
summary_class['InitialPreds'] = initial_preds
summary_class

Ha WeekTransplanted               Ranch  \
                          sum              min max nunique nunique   
Class Type                                                           
BSUF  Uva Amarilla     2.4745                6  28      12       6   
      Uva Roja        27.9135                4  31      19       6   
CHE   Cherry Naranja   0.6529               12  30       9       5   
      Cherry Rojo     18.2069                4  31      15       6   
      Cherry Rosa      3.7839                9  29      13       6   
      Pera Amarilla    0.6037               14  24       6       6   
      Pera Roja        0.4607               11  24       5       4   

                      KilosReceived     FinalPreds   InitialPreds  
                                                                   
Class Type                                                         
BSUF  Uva Amarilla     26024.000000   25244.899242   34496.524660  
      Uva Roja        678726.062500  679637.719629  735640.579411  
CHE   Cherry Naranja    7677.000000    8210.960366   11366.974544  
      Cherry Rojo     429096.031250  419290.771146  457121.058864  
      Cherry Rosa      48863.000000   46821.822337   63760.054190  
      Pera Amarilla     6097.999512    6510.656533    8248.894319  
      Pera Roja         3880.000000    4197.148301    3288.950689

by class

In [30]:
classes = ['BSUF','CHE']
classes_idx = table.get_indices('Class',classes,idx_dict)
class_transplant_weeks = transplant_weeks[classes_idx]

In [31]:
year_preds_by_class, year_actuals_by_class = table.filter_preds(year_preds,year_actuals,'Class',classes,idx_dict)
preds_by_class, actuals_by_class = table.filter_preds(preds,actuals,'Class',classes,idx_dict)

In [32]:
fig6 = graphs.graph_harvest_stacked(year_preds_by_class,year_actuals_by_class,classes)
fig6b = graphs.graph_harvest_stacked(preds_by_class,actuals_by_class,classes)
fig6.show()
fig6b.show()

by type

In [33]:
types = ['Uva Roja','Cherry Rosa']
types_idx = table.get_indices('Type',types,idx_dict)
type_transplant_weeks = transplant_weeks[types_idx]


In [34]:
year_preds_by_type, year_actuals_by_type = table.filter_preds(year_preds,year_actuals,'Type',types,idx_dict)

In [42]:
fig7 = graphs.graph_harvest_stacked(year_preds_by_type,year_actuals_by_type,types)
fig7.show()

by ranch, all 3 and table

In [36]:
ranches = ['OAP','VAP']
ranches_idx = table.get_indices('Ranch',ranches,idx_dict)
ranch_transplant_weeks = transplant_weeks[ranches_idx]

In [37]:
year_preds_by_ranch, year_actuals_by_ranch = table.filter_preds(year_preds,year_actuals,'Ranch',ranches,idx_dict)

In [38]:
fig8 = graphs.graph_harvest_stacked(year_preds_by_ranch,year_actuals_by_ranch,ranches)
fig8.show()